# Импорты и функции

In [60]:
import requests
from datetime import datetime
import telebot
from telebot import types
import sqlite3
import random

import numpy as np
import pandas as pd
import psutil
from IPython.display import Image

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

import kaleido
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objects as go
import collections

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [230]:
def candles(data, type2 = False):
    #declare figure
    fig = go.Figure()

    #Candlestick
    fig.add_trace(go.Candlestick(x=data.index,
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close'], name = 'market data'))

    # Add titles
    """fig.update_layout(
        #title='Uber live share price evolution',
        yaxis_title='Курс акций в долларах')
"""
    # Делаем кнопки для навигации
    """ fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                #dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1month", step="month", stepmode="backward"),
                dict(step="all")
            ])
        )
    ) """

    #настраиваем размер графика
    fig.update_layout(autosize=True,
                      width=1000,
                      height=700,)

    #убираем разрывы меежду данными. Чтобы график был сплошной. То есть убираем выходные и часы, когда торговля не идет
    if type2:    
        fig.update_xaxes(
                rangeslider_visible=False,
                rangebreaks=[
                    # NOTE: Below values are bound (not single values), ie. hide x to y
                    dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                    #dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                    # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
                ]
            )
    else:
        fig.update_xaxes(
                rangeslider_visible=False,
                rangebreaks=[
                    # NOTE: Below values are bound (not single values), ie. hide x to y
                    dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                    dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                    # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
                ]
            )

    #Show
    #fig.show()
    
    return fig

In [232]:
def current_price(symbol):
    data = yf.download(tickers=symbol, period='1m', interval='1m')
    
    return data['Close'][-1]

In [233]:
def get_quotes(ticker, period, interval):
    data = yf.download(tickers= ticker, period= period, interval= interval)
    
    if data.size != 0:
        return data
    else:
        return 'неправильные данные'

# черновик

In [234]:
# fig.to_image(format="png", engine="kaleido")

In [235]:
# fig = candles(get_quotes('TSLA', '10d', '15m'))

# Бот

In [236]:
bot = telebot.TeleBot("1834334715:AAEtTI6RuWqg7zMW0uCPJWb5_DfRfQ7UdtI")

In [237]:
# result_photo = candles(get_quotes('TSLA', '10d', '1h')).to_image(format="png", engine="kaleido")

In [239]:
@bot.message_handler(content_types=['text'])
def responser(message, text='а там армяне в нарды играют'):
    try:
        ticker, period, interval = message.text.split()
        if interval in ["1m","2m","5m","15m","30m","60m","90m","1h"]:
            result_photo = candles(get_quotes(ticker, period, interval)).to_image(format="png", engine="kaleido")
            bot.send_photo(message.chat.id, result_photo)
        else:
            result_photo = candles(get_quotes(ticker, period, interval), type2 = True).to_image(format="png", engine="kaleido")
            bot.send_photo(message.chat.id, result_photo)
    except:
        bot.send_message(message.chat.id, text = 'Укажите другие данные')

In [240]:
bot.polling(none_stop=True)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- YNDX: 1m data not available for startTime=1622735493 and endTime=1623599493. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            
Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo           
        

In [252]:
current_price('TSLA')


[*********************100%***********************]  1 of 1 completed


609.8900146484375

# Используем апишки для получения новостей

Я написал по две версии функций для разных апи. Кмк полигон (вторые версии) получше

In [213]:
goog_token = "ef44e796dd82495d9b0ac4ed58354d70"
hub_token = "c33nijqad3i8edlc7t50"
polygon_token = "ZrbA9bOMxg8Nf2qyV14nzV1DLk65mFip"

In [214]:
#получение одной новости из финхаба
def get_news(ticker):
    global hub_token
    today = str(datetime.date(datetime.now()))
    r = requests.get(f'https://finnhub.io/api/v1/company-news?symbol={ticker}&from={today}&to={today}&token={token}')
    headline = r.json()[0]['headline']
    url = r.json()[0]['url']
    return (headline, url)

In [215]:
#получение одной новости из полигона
def get_news2(ticker): 
    global polygon_token
    url = (f'https://api.polygon.io/v2/reference/news?limit=10&order=descending&sort=published_utc&ticker={ticker}&published_utc.gte=2021-04-26&apiKey={polygon_token}')
    response = requests.get(url).json()['results']
    headline = response[0]['title']
    url = response[0]['article_url']
    return (headline, url)

In [216]:
#получение списка всех новостей из финхаба (заголовки)
def get_news_headlines(ticker):
    global hub_token
    today = str(datetime.date(datetime.now()))
    r = requests.get(f'https://finnhub.io/api/v1/company-news?symbol={ticker}&from={today}&to={today}&token={token}')
    r = r.json()
    #print(r)
    a = []
    for i in r:
        a.append(i['headline'])
    return a

In [217]:
#получение списка всех новостей из полигона (заголовки)
def get_news_headlines2(ticker):
    global polygon_token
    url = (f'https://api.polygon.io/v2/reference/news?limit=10&order=descending&sort=published_utc&ticker={ticker}&published_utc.gte=2021-04-26&apiKey={polygon_token}')
    r = requests.get(url).json()['results']
    a = []
    for i in r:
        a.append(i['title'])
    return a

In [218]:
get_news("MSFT")

('2021 Year-To-Date Percentage Returns Of 8 Largest Stocks - Saturday, June 12',
 'https://finnhub.io/api/news?id=7093c3c36749d924d1aebf16eeb50241b0518fb15520d2e2a37d534b55baa74a')

In [219]:
get_news2("MSFT")

('Qualcomm Offers Help to Arm Holdings If NVIDIA Acquisition Is Blocked',
 'https://www.fool.com/investing/2021/06/14/qualcomm-offers-help-to-arm-holdings-if-nvidia-acq/')

In [220]:
get_news_headlines("MSFT")

['2021 Year-To-Date Percentage Returns Of 8 Largest Stocks - Saturday, June 12',
 'Dow Jones Futures Edge Higher: Apple Vs. Microsoft Vs. Google; How To Be A Big Stock Market Winner',
 'Microsoft bolsters video game line-up as Xbox turns 20']

In [225]:
get_news_headlines2("MSFT")

['Qualcomm Offers Help to Arm Holdings If NVIDIA Acquisition Is Blocked',
 "Can AMD Help Microsoft Beat Sony PS5's Scorching Sales?",
 '3 Enterprise Communications Stocks in Focus as Hybrid Work Revs Up',
 'Google Intensifies Microsoft, Zoom, Slack Competition Via Workspace Suite Beef Up: Bloomberg',
 'Microsoft (MSFT) to Bring 27 New Titles to Xbox Game Pass',
 'The Zacks Analyst Blog Highlights: Microsoft, Nike, Novo Nordisk, Caterpillar and General Electric',
 "What to Watch at Nintendo's E3 Presentation",
 'Should Invesco Dividend Achievers ETF (PFM) Be on Your Investing Radar?',
 'This Is Hands Down the Safest Robinhood Stock You Can Buy Right Now',
 'Can Zoom Video Stock Reach $600 Again?']